In [40]:
import os
import re
import pickle

import wget
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# Download all RFC files

In [43]:
def download_rfc(base_url, outdir, total_rfc):
    """
    """
    # Create output directory if not exists.
    os.makedirs(outdir, exist_ok=True)

    # Download all RFCs.
    errors = []
    for i in tqdm(range(total_rfc)):
        # Create the url.
        url = base_url + str(i+1) + '.txt'

        # Download page.
        try:
            wget.download(url, outdir)
        except Exception as e:
            errors.append(i+1)
            print("RFC {}: HTTP Error 404 - Not Found.".format(i+1))
            
    # Save 404 errors.
    with open(os.path.join(outdir, 'errors404'), 'wb') as out:
        pickle.dump(errors, out)

In [44]:
download_rfc(base_url='https://tools.ietf.org/rfc/rfc', 
             outdir='/raid/antoloui/Master-thesis/_data/search/rfc', 
             total_rfc=8774)

RFC 8: HTTP Error 404 - Not Found.
RFC 9: HTTP Error 404 - Not Found.



# Create clean dataframe from RFC index

In [55]:
def concat_rfc_lines(lines):
    """
    Given a list of lines where a same RFC is described on multiple lines, concat
    the lines describing the same RFC.
    """
    rfc_lines = []
    current_rfc = ''
    for line in lines:
        if line.startswith('RFC'):
            rfc_lines.append(current_rfc)  # End of previous RFC, append it to list.
            current_rfc = line  # Get beginning of new rfc.
        else:
            current_rfc += line
    return rfc_lines


def remove_multiple_spaces(text):
    """
    Given a string, replace all multiple spaces in it by a single space.
    """
    text = re.sub('\s{2,}', ' ', text)
    text = text.lstrip().rstrip()  # Remove whitespaces in beginning or end of string.
    return text
    

def get_rfc_lines(page, errors_filepath):
    """
    Given the result of an url request, get the text of interest.
    """
    # Load the page with BeautifulSoup.
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get the text of interest (the index is in <pre>...</pre>).
    body = soup('pre')[0]
    
    # Get plain text.
    content = body.get_text() 
    
    # Remove all text before the line beginning by 'RFC1' (beginning of the index).
    content = content.split('RFC1 ')[1]
    content = 'RFC1 ' + content
    
    # Split raw text to lines.
    lines = content.splitlines()
    lines = [line for line in lines if line != ''] # remove empty lines.
    
    # Concat lines describing the same RFC.
    rfc_lines = concat_rfc_lines(lines)
    
    # Remove multiple spaces.
    rfc_lines = [remove_multiple_spaces(line) for line in rfc_lines]
    
    # Remove all 'Non Issued' RFCs.
    rfc_lines = [line for line in rfc_lines if 'Not Issued' not in line]
    
    # Remove RFC that were not found during download.
    with open(errors_filepath, "rb") as f:
        errors = pickle.load(f)
    rfc_lines = [line for line in rfc_lines if int(line.split(' ', 1)[0].split('RFC')[1]) not in set(errors)]
    
    return rfc_lines[1:]


def create_dataframe(rfc_lines):
    """
    Given the lines describing each RFC, create a dataframe.
    """
    # Init lists.
    names = []
    titles = []
    authors = []
    dates = []
    formats = []
    obsolotes = []
    obsoloted = []
    updates = []
    updated = []
    also = []
    status = []
    dois = []
    
    # Process each line.
    for i, line in enumerate(tqdm(rfc_lines)):
        
        # Get all attributes within brackets.
        brackets = re.findall(r"\((.*?)\)", line)

        # Get individual attributes.
        form = None
        obs = None
        obs_by = None
        up = None
        up_by = None
        al = None
        stat = None
        doi = None
        for att in brackets:
            if att.startswith('Format: '):
                form = att.split('Format: ')[1]
            elif att.startswith('Obsolotes '):
                obs = att.split('Obsolotes ')[1]
            elif att.startswith('Obsoleted by '):
                obs_by = att.split('Obsoleted by ')[1]
            elif att.startswith('Updates '):
                up = att.split('Updates ')[1]
            elif att.startswith('Updated by '):
                up_by = att.split('Updated by ')[1]
            elif att.startswith('Also '):
                al = att.split('Also ')[1]
            elif att.startswith('Status: '):
                stat = att.split('Status: ')[1]
            elif att.startswith('DOI: '):
                doi = att.split('DOI: ')[1]
        line = line.split('(Format')[0].rstrip()  # Remove bracket attributes from the line.

        # Get the date of publication.
        split_line = line.split(".")
        split_line = [l for l in split_line if l != '']
        date = split_line[-1].lstrip()
        line = line.replace(date + '.', '')  # Remove date from line.

        # Get name of RFC.
        name = line.split()[0]
        line = line.replace(name, '')  # Remove name from line.

        # Get title of RFC.
        title = line.split('.')[0].lstrip()
        line = line.replace(title + '.', '')  # Remove title from line.

        # Get authors.
        aut = line.lstrip().rstrip()[:-1]

        # Append all info to corresponding list.
        names.append(name)
        titles.append(title)
        authors.append(aut)
        dates.append(date)
        formats.append(form)
        obsolotes.append(obs)
        obsoloted.append(obs_by)
        updates.append(up)
        updated.append(up_by)
        also.append(al)
        status.append(stat)
        dois.append(doi)

    # Create dataframe.
    d = {'Name':names,
         'Ttile':titles,
         'Authors':authors,
         'Date':dates,
         'Formats':formats,
         'Obsolotes':obsolotes,
         'Obsoloted_by':obsoloted,
         'Updates':updates,
         'Updated_by':updated,
         'Also_FYI':also,
         'Status':status,
         'DOI':dois}
    df = pd.DataFrame(d)
    return df
        

def main(url, errors_filepath):
    """
    """
    # Download the page.
    page = requests.get(url)
    
    # Get all RFC lines.
    rfc_lines = get_rfc_lines(page, errors_filepath)
    
    # Create dataframe.
    df = create_dataframe(rfc_lines)
    return df

In [53]:
df = main(url='https://tools.ietf.org/rfc/index', 
          errors_filepath='/raid/antoloui/Master-thesis/_data/search/rfc/errors404')
df.head()